# Decompose the IVT field into a background and a anomaly component


This notebook will apply the Top-hat by Reconstruction (THR) algorithm on the computed IVT from the previous step, and the outputs will be used for AR appearance detection in the next notebook.

In production you can use the `compute_thr_singlefile.py` or `compute_thr_multifile.py` for this step.


## Input data

* `ivt_m1-60_6_1984_crop.nc`: Integrated Vapor Transport (IVT), in kg/m/s.

## Steps

1. Make sure you have successfully run the previous notebook.
2. Execute the following code blocks in sequence.


## Results

* `ivt_m1-60_6_1984_crop-minimal-rec-ano-kernel-t16-s6.nc`: decomposition of IVT using the THR algorithm, into a reconstruction component (`ivt_rec`) and the anomaly component (`ivt_ano`), both in kg/m/s.


## Set global parameters

Set the path of the input netCDF file, the id of the IVT data, the folder to save the outputs.

`LAT1` and `LAT2` selects the latitude range to process.

`SHIFT_LON` shifts the data along the x-dimension by 80 degrees so the Pacific and Atlantic oceans are centered.

`KERNEL` is the most important parameter. It is a list/tuple of ints, specifying the shape of the kernel (aka structuring element) during the gray erosion process. Read the paper for more detailed explanations. A rule-of-thumb is that you should set the kernel size to be consistent with the typical synoptic spatio-temporal scales.

E.g.

Assuming the data is 6-hourly, at $0.75 \times 0.75$ degree longitude/latitude resolution. We want the temporal scale to be about a week, therefore $t = 16$. This is because 16 time steps of 6-hourly data is 4 days, and the **total** length is $16 \times 2 + 1 = 33$ time steps, giving $33 \times 6 = 108 hrs$, about a week's time. The extra added time step is to ensure an odd numbered kernel length.

Similarly, for the spatial scale we want it to be about $1000 km$. With $s=6$ and a horizontal resolution of $0.75 \times 0.75$ degrees, this gives a length of $6 \times 2 + 1 = 13$ grid cells, about $1140 \, km$.

Combing the above: `KERNEL=[16, 6, 6]`.

If your input data have $0.25 \times 0.25$ degree horizontal resolution, then $s$ should be scaled to $18$ to be consistent.

In [ ]:
%matplotlib inline
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import cdms2 as cdms
import MV2 as MV
import numpy as np
from skimage import morphology
from utils import funcs
from compute_thr_singlefile import filterData

## Parameters

#-----------IVT data----------------------
IVT_FILE=os.path.join('.', 'ivt_m1-60_6_1984_crop.nc')
VARIN='ivt'          # data id in nc file

LAT1=0; LAT2=90      # degree, latitude domain

#-------Structuring element for erosion (E)-------
KERNEL=[16,6,6]   # half length of time (time steps), and half length of spatial (number of grids)

SHIFT_LON=80  # shift longitudinally to center Pacific and Altantic

#------------------Output folder------------------
OUTPUTDIR=os.path.abspath('.')

Then the input data are read in, crop the desired latitude, and shift the longitude.

In [ ]:
if not os.path.exists(OUTPUTDIR):
    os.makedirs(OUTPUTDIR)

#-----------Read in data----------------------
var=funcs.readVar(IVT_FILE, 'ivt')

#-----------------Shift longitude-----------------
var=var(latitude=(LAT1, LAT2))
var=var(longitude=(SHIFT_LON,SHIFT_LON+360))

print('var.shape=', var.shape)

Perform the THR process. This might take a couple of minutes (depending on your machine of cause). And save the output to the same directory as this notebook file.

In [ ]:
#----------------------Do THR----------------------
ivt, ivtrec, ivtano=filterData(var, KERNEL)

#--------Save------------------------------------
fname=os.path.split(IVT_FILE)[1]
file_out_name='%s-minimal-rec-ano-kernel-t%d-s%d.nc'\
        %(os.path.splitext(fname)[0], KERNEL[0], KERNEL[1])

abpath_out=os.path.join(OUTPUTDIR,file_out_name)
print('\n### <testrotatingfilter>: Saving output to:\n',abpath_out)
fout=cdms.open(abpath_out,'w')
fout.write(ivt,typecode='f')
fout.write(ivtrec,typecode='f')
fout.write(ivtano,typecode='f')
fout.close()

Create some plots using results at an arbitrary time point.

In [ ]:
#------- Plot a figure -----------
import matplotlib.pyplot as plt
from utils import plot

figure=plt.figure(figsize=(7,10),dpi=100)
idx=100  # select the 101th time step from the beginning
time_str=ivt.getTime().asComponentTime()[idx]

plot_vars=[ivt[idx], ivtrec[idx], ivtano[idx]]
iso=plot.Isofill(plot_vars, 12, 1, 1, min_level=0, qr=0.001)
titles=['IVT (=IVT_rec + IVT_ano)', 'IVT_rec', 'IVT_ano']

for ii, vii in enumerate(plot_vars):
    axii=figure.add_subplot(3,1,ii+1)
    
    plot.plot2(vii, iso, axii,
            title='%s, %s' %(str(time_str), titles[ii]),
            legend='local',
            projection='cyl',
            fix_aspect=False)

figure.show()